## Выбросы

In [ ]:
def outliers(jeans):
    season_data = pd.DataFrame({'ds': jeans.index, 'y': jeans.values})
    
    # Создание модели Prophet с уменьшенной чувствительностью к выбросам
    model = Prophet(yearly_seasonality=True)
    
    model.fit(season_data)
    
    # Прогнозирование на тех же временных точках (исторический прогноз)
    forecast = model.predict(season_data)
    
    # Вычисление остатков
    residuals = season_data['y'] - forecast['yhat']
    
    # Расчёт стандартного отклонения остатков
    std_res = residuals.std()
    
    # Определение порога для выбросов: 2.48 * стандартное отклонение
    threshold = 5 * std_res
    
    # Нахождение индексов точек, где абсолютное значение остатка больше порога
    outlier_mask = np.abs(residuals) > threshold
    outlier_indices = residuals.index[outlier_mask]
        
    # Коррекция выбросов: для каждой точки, где наблюдается выброс,
    # вычитаем остаток из исходного значения (результат становится равен предсказанному значению)
    season_data_corrected = season_data.copy()
    season_data_corrected.loc[outlier_indices, 'y'] = season_data_corrected.loc[outlier_indices, 'y'] - residuals.loc[outlier_indices]
    
    jeans = season_data_corrected
    jeans = jeans.set_index('ds')['y']
    return jeans

## Структурные сдвиги

In [ ]:
def deviation_generating(jeans, seasonality, adv_actual, cost_actual, leftovers):
    common_index = jeans.index.intersection(seasonality.index).intersection(adv_actual.index).intersection(cost_actual.index)
    
    # Обрезаем все временные ряды по общему индексу
    deviation_jeans = jeans.loc[common_index]
    deviation_seasonality = seasonality.loc[common_index]
    deviation_advertising = adv_actual.loc[common_index]
    deviation_leftovers = leftovers.loc[common_index]
    deviation_cost = cost_actual.loc[common_index]
    
    # Основной временной ряд (например, заказы)
    signal = deviation_jeans.values
    
    # Преобразуем регрессоры в нужный формат
    X_seasonality = deviation_seasonality.values.reshape(-1, 1)
    X_advertising = deviation_advertising.values.reshape(-1, 1)
    X_leftovers = deviation_leftovers.values.reshape(-1, 1)
    X_cost = deviation_cost.values.reshape(-1, 1)
    
    # Объединяем все регрессоры в одну матрицу признаков
    X = np.hstack([X_seasonality, X_advertising, X_leftovers, X_cost])
    
    # Обучаем линейную регрессию с ограничением на положительные коэффициенты
    model = LinearRegression(positive=True)
    model.fit(X, signal)
    trend = model.predict(X)
    
    residuals = signal - trend
    
    # Поиск структурных сдвигов в остатках
    algo = rpt.Binseg(model="l2").fit(residuals)
    breakpoints = algo.predict(pen=7000)  # Параметр pen можно корректировать для точности
    
    def generate_regressor_series(data, breakpoints):
        """
        Генерирует булевый регрессорский ряд на основе списка точек разбиения.
        
        :param data: Pandas Series, исходный временной ряд
        :param breakpoints: List[int], список индексов разбиения
        :return: Pandas Series, расширенный ряд булевых значений
        """
        L = len(data)
        extended_length = L + 31  # Расширяем на 31 точку
        regressor = np.zeros(extended_length, dtype=bool)
        
        # Заполняем булевый массив по сегментам
        segments = [0] + breakpoints + [extended_length]
        for i in range(0, len(segments) - 1, 2):
            regressor[segments[i]:segments[i+1]] = True
        
        # Определяем частоту временного ряда
        freq = pd.infer_freq(data.index)
        if freq is None:
            freq = 'D'
        
        # Генерируем расширенный индекс
        extended_index = pd.date_range(start=data.index[0], periods=extended_length, freq=freq)
        
        return pd.Series(regressor, index=extended_index)
    
    # Пример использования:
    breakpoints_sep = breakpoints[:-1] # [203, 351]
    regressor_series = generate_regressor_series(jeans, breakpoints_sep)
    deviation_regressor = regressor_series
    
    deviation_regressor = ~deviation_regressor
    return deviation_regressor

## Prophet

In [ ]:
import pandas as pd
from prophet import Prophet

def prophet_fn(
    jeans: pd.Series,
    advertising: pd.Series,
    seasonality: pd.Series,
    deviation_regressor: pd.Series,
    leftovers: pd.Series,
    cost: pd.Series,
    n_future: int
) -> pd.Series:

    series_list = [jeans, advertising, seasonality, deviation_regressor, leftovers, cost]
    # начальные даты (учитываем только непустые значения)
    start_dates = [s.dropna().index.min() for s in series_list]
    # последние доступные даты
    end_dates   = [s.index.max() for s in series_list]
    common_start = max(start_dates)
    common_end   = min(end_dates)

    # 2) Создаём равномерный индекс по дням для исторического периода
    hist_idx = pd.date_range(start=common_start, end=common_end, freq='D')

    # 3) Рейнжируем и заполняем пропуски (регрессоры — заполнением вперед, потом нулями)
    jeans_hist       = jeans.reindex(hist_idx)
    advertising_hist = advertising.reindex(hist_idx).fillna(method='ffill').fillna(0)
    seasonality_hist = seasonality.reindex(hist_idx).fillna(method='ffill').fillna(0)
    deviation_hist   = deviation_regressor.reindex(hist_idx).fillna(method='ffill').fillna(0)
    leftovers_hist   = leftovers.reindex(hist_idx).fillna(method='ffill').fillna(0)
    cost_hist        = cost.reindex(hist_idx).fillna(method='ffill').fillna(0)

    # 4) Формируем DataFrame для обучения и удаляем строки с NaN в y
    prophet_df = pd.DataFrame({
        'ds': hist_idx,
        'y': jeans_hist.values,
        'regressor': advertising_hist.values,
        'seasonality': seasonality_hist.values,
        'deviation': deviation_hist.values,
        'leftovers': leftovers_hist.values,
        'cost': cost_hist.values
    }).dropna(subset=['y'])

    # 5) Инициализация и обучение модели
    model = Prophet(weekly_seasonality=True, mcmc_samples=800)
    for reg in ['regressor', 'seasonality', 'deviation', 'leftovers', 'cost']:
        model.add_regressor(reg)
    model.fit(prophet_df)

    # 6) Построение будущего датафрейма
    future = model.make_future_dataframe(periods=n_future, freq='D')
    future_idx = future['ds']

    # 7) Подготавливаем регрессоры для будущего (заполнение аналогично историческому)
    future['regressor']  = advertising.reindex(future_idx).fillna(method='ffill').fillna(0).values
    future['seasonality'] = seasonality.reindex(future_idx).fillna(method='ffill').fillna(0).values
    future['deviation']   = deviation_regressor.reindex(future_idx).fillna(method='ffill').fillna(0).values
    future['leftovers']   = leftovers.reindex(future_idx).fillna(method='ffill').fillna(0).values
    future['cost']        = cost.reindex(future_idx).fillna(method='ffill').fillna(0).values


    # 8) Прогноз
    forecast = model.predict(future)

    # Возвращаем только новые даты после исторического конца
    result = forecast.set_index('ds')['yhat']
    return result[result.index > common_end]


## SARIMA

In [5]:
import pandas as pd
import statsmodels.api as sm

def sarima_fn(
    advertising: pd.Series,
    seasonality: pd.Series,
    jeans: pd.Series,
    deviation_regressor: pd.Series,
    leftovers: pd.Series,
    cost: pd.Series,
    p: int,
    d: int,
    q: int,
    P: int,
    D: int,
    Q: int,
    m: int,
    n_future: int,
    need_summary: bool
) -> pd.Series:

    deviation_regressor = deviation_regressor.astype(int)
    leftovers = leftovers.astype(int)
    
    # 1) Определяем общий период, где есть данные во всех сериях
    series_list = [jeans, advertising, seasonality, deviation_regressor, leftovers, cost]
    start_dates = [s.dropna().index.min() for s in series_list]
    end_dates = [s.index.max() for s in series_list]
    common_start = max(start_dates)
    common_end = min(end_dates)

    # 2) Создаем единый дневной индекс
    hist_idx = pd.date_range(start=common_start, end=common_end, freq='D')

    # 3) Реиндексация и заполнение
    jeans_hist = jeans.reindex(hist_idx)
    ad_hist = advertising.reindex(hist_idx).fillna(method='ffill').fillna(0)
    seas_hist = seasonality.reindex(hist_idx).fillna(method='ffill').fillna(0)
    dev_hist = deviation_regressor.reindex(hist_idx).fillna(method='ffill').fillna(0)
    left_hist = leftovers.reindex(hist_idx).fillna(method='ffill').fillna(0)
    cost_hist = cost.reindex(hist_idx).fillna(method='ffill').fillna(0)

    # 4) Формируем DataFrame и убираем даты без y
    df_train = pd.DataFrame({
        'jeans': jeans_hist.values,
        'advertising': ad_hist.values,
        'seasonality': seas_hist.values,
        'deviation': dev_hist.values,
        'leftovers': left_hist.values,
        'cost': cost_hist.values
    }, index=hist_idx).dropna(subset=['jeans'])

    end_train = df_train.index.max()
    y_train = df_train['jeans']
    exog_train = df_train[['advertising','seasonality','deviation','leftovers','cost']]

    # 5) Инициализация и обучение модели SARIMAX
    model = sm.tsa.statespace.SARIMAX(
        y_train,
        exog=exog_train,
        order=(p, d, q),
        seasonal_order=(P, D, Q, m),
        enforce_stationarity=False,
        enforce_invertibility=False
    )
    results = model.fit(disp=False)
    print(f"Обучена модель SARIMA({p},{d},{q})x({P},{D},{Q},{m}) AIC={results.aic:.1f}")

    # 6) Подготовка будущих экзогенных регрессоров
    start_fc = end_train + pd.Timedelta(days=1)
    fc_idx = pd.date_range(start=start_fc, periods=n_future, freq='D')
    exog_fc = pd.DataFrame({
        'advertising': advertising.reindex(fc_idx).fillna(method='ffill').fillna(0).values,
        'seasonality': seasonality.reindex(fc_idx).fillna(method='ffill').fillna(0).values,
        'deviation': deviation_regressor.reindex(fc_idx).fillna(method='ffill').fillna(0).values,
        'leftovers': leftovers.reindex(fc_idx).fillna(method='ffill').fillna(0).values,
        'cost': cost.reindex(fc_idx).fillna(method='ffill').fillna(0).values
    }, index=fc_idx)

    if need_summary:
        return results.summary()

    # 7) Прогноз
    forecast = results.get_forecast(steps=n_future, exog=exog_fc)
    return pd.Series(forecast.predicted_mean, index=fc_idx)


## Bayesian Structure TimeSeries

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

def bsts_fn(
    jeans,
    advertising,
    seasonality,
    deviation_regressor,
    leftovers,
    cost,
    forecast_periods=31,
    seed=123,
    burn=1000,
    samples=10000
):
    deviation_regressor = deviation_regressor.astype(int)
    leftovers = leftovers.astype(int)

    # 1) Определяем общий период, где есть данные во всех сериях
    series_list = [jeans, advertising, seasonality, deviation_regressor, leftovers, cost]
    start_dates = [s.dropna().index.min() for s in series_list]
    end_dates = [s.index.max() for s in series_list]
    common_start = max(start_dates)
    common_end = min(end_dates)

    # 2) Создаем единый дневной индекс
    hist_idx = pd.date_range(start=common_start, end=common_end, freq='D')

    # 3) Реиндексация и заполнение
    jeans_hist = jeans.reindex(hist_idx)
    regressor_hist = advertising.reindex(hist_idx).fillna(method='ffill').fillna(0)
    seasonality_hist = seasonality.reindex(hist_idx).fillna(method='ffill').fillna(0)
    deviation_hist = deviation_regressor.reindex(hist_idx).fillna(method='ffill').fillna(0)
    leftovers_hist = leftovers.reindex(hist_idx).fillna(method='ffill').fillna(0)
    cost_hist = cost.reindex(hist_idx).fillna(method='ffill').fillna(0)
    
    df_train = pd.DataFrame({
        'jeans': jeans_hist.values,
        'advertising': regressor_hist.values,
        'seasonality': seasonality_hist.values,
        'deviation': deviation_hist.values,
        'leftovers': leftovers_hist.values,
        'cost': cost_hist.values
    }, index=hist_idx).dropna(subset=['jeans'])

    jeans_hist = df_train['jeans']
    exog_hist = df_train[['advertising', 'seasonality', 'deviation', 'leftovers', 'cost']]

    # Обучаем регрессионную модель
    exog_with_const = sm.add_constant(exog_hist, has_constant='add')
    reg_model = sm.OLS(jeans_hist, exog_with_const).fit()
    residuals = jeans_hist - reg_model.predict(exog_with_const)

    # Байесовская модель
    bayes_uc = BayesianUnobservedComponents(
        response=residuals,
        level=True,
        stochastic_level=True,
        trend=True,
        stochastic_trend=True,
        trig_seasonal=((7, 0),),
        stochastic_trig_seasonal=(True,),
        seed=seed
    )
    bayes_uc.sample(samples)

    # 5) Будущие значения экзогенных переменных
    last_hist_date = df_train.index.max()
    future_dates = pd.date_range(start=last_hist_date + pd.Timedelta(days=1), periods=forecast_periods, freq='D')

    exog_future = pd.DataFrame({
        'advertising': advertising.reindex(future_dates).fillna(method='ffill').fillna(0).values,
        'seasonality': seasonality.reindex(future_dates).fillna(method='ffill').fillna(0).values,
        'deviation': deviation_regressor.reindex(future_dates).fillna(method='ffill').fillna(0).values,
        'leftovers': leftovers.reindex(future_dates).fillna(method='ffill').fillna(0).values,
        'cost': cost.reindex(future_dates).fillna(method='ffill').fillna(0).values
    }, index=future_dates)

    exog_future_with_const = sm.add_constant(exog_future, has_constant='add')
    exog_future_with_const = exog_future_with_const[reg_model.params.index]
    regression_forecast = reg_model.predict(exog_future_with_const)

    # Прогноз остатков
    forecast_samples, _ = bayes_uc.forecast(num_periods=forecast_periods, burn=burn)
    forecast_resid_mean = np.mean(forecast_samples, axis=0).squeeze()
    forecast_resid_l95 = np.quantile(forecast_samples, 0.025, axis=0).squeeze()
    forecast_resid_u95 = np.quantile(forecast_samples, 0.975, axis=0).squeeze()

    # Финальный прогноз
    forecast_mean = regression_forecast + forecast_resid_mean
    forecast_l95 = regression_forecast + forecast_resid_l95
    forecast_u95 = regression_forecast + forecast_resid_u95

    forecast_series = pd.Series(forecast_mean, index=future_dates)
    return forecast_series, forecast_l95, forecast_u95


## Генерация параметров

In [27]:
from statsmodels.tsa.stattools import acf, pacf, pacf_yw, pacf_ols
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

def acf_pacf_to_sarima(series, nlags=50, alpha=0.05, look_at=1, plot=False):
    """PACF using Yule-Walker method."""
    n = len(series)
    z = abs(np.percentile(np.random.normal(size=100000), 100 * (1 - alpha / 2)))
    conf_int = z / np.sqrt(n)


    acf_vals = acf(series, nlags=nlags)
    pacf_vals = pacf_yw(series, nlags=nlags, method='mle')

    p = 0
    q = 0

    for val in pacf_vals[::look_at]:
        if abs(val) > conf_int:
            p += 1
        else:
            break

    for val in acf_vals[::look_at]:
        if abs(val) > conf_int:
            q += 1
        else:
            break

    p = min(6, p - 1)
    q = min(6, q - 1)

    if plot:
        fig, axes = plt.subplots(2, 1, figsize=(10, 8))
        plot_acf(series, lags=nlags, alpha=alpha, ax=axes[0])
        axes[0].set_title("ACF Plot")
        axes[1].stem(range(len(pacf_vals)), pacf_vals)
        axes[1].set_title("PACF Plot (Yule-Walker)")
        axes[1].axhline(conf_int, linestyle='--', color='red')
        axes[1].axhline(-conf_int, linestyle='--', color='red')
        plt.tight_layout()
        plt.show()

    return [p, q]

def is_stationary(series):
    if series.empty:
        return False
    adf_p = adfuller(series.dropna())[1]
    kpss_p = kpss(series.dropna(), regression='c')[1]
    return adf_p < 0.05 and kpss_p > 0.05

def params_generating(jeans):
    series = jeans.copy()
    
    if is_stationary(jeans.diff()):
        params = acf_pacf_to_sarima(series.diff().dropna())
        d = 1
    else:
        params = acf_pacf_to_sarima(series.diff().diff().dropna())
        d = 2
    p, q = params[:]

    if is_stationary(jeans.diff(periods=7)):
        params = acf_pacf_to_sarima(series.diff(periods=7).dropna(), look_at=7)
        D = 1
    else:
        params = acf_pacf_to_sarima(series.diff(periods=7).diff(periods=7).dropna(), look_at=7)
        D = 2
    P, Q = params[:]
    return p, d, q, P, D, Q

## Параметры ансамбля

In [ ]:
def ensemble_params_generating(n_future, jeans, P):
    n_iter = min(3, (len(jeans) // n_future))
    return n_iter

## Определение весов ансамбля

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from functools import reduce

def compute_model_weights(
    y: pd.Series,
    adv_actual: pd.Series,
    seasonality: pd.Series,
    deviation_regressor: pd.Series,
    leftovers: pd.Series,
    cost_actual: pd.Series,
    sarima_fn,
    prophet_fn,
    bsts_fn,
    n_future: int = 31,
    p: int = 1,
    d: int = 0,
    q: int = 1,
    P: int = 0,
    D: int = 0,
    Q: int = 0,
    m: int = 7,
    use_box_cox: bool = True,
    alpha: float = 0.5,
    offset: float = None,
    n_splits: int = 3
) -> dict:

    def scale(series, train_idx):
        train = series.iloc[train_idx]
        mean, std = train.mean(), train.std()
        std = std if std else 1  # Avoid division by zero
        return (series - mean) / std

    def safe_log1p(series, offset_val):
        return np.log1p(series + offset_val)

    def inverse_transform(preds, offset_val):
        vals = np.expm1(preds) - offset_val if use_box_cox else preds
        vals = np.where(np.isfinite(vals), vals, np.nan)
        return np.nan_to_num(vals, nan=0.0, posinf=0.0, neginf=0.0)

    # Prepare and align data
    all_series = [
        y, adv_actual, seasonality,
        deviation_regressor, leftovers, cost_actual
    ]

    common_index = reduce(lambda a, b: a.intersection(b), [s.dropna().index for s in all_series])
    common_index = common_index.sort_values()

    # Clip everything to common index
    y = y.loc[common_index]
    adv_actual = adv_actual.loc[common_index]
    seasonality = seasonality.loc[common_index]
    deviation_regressor = deviation_regressor.loc[common_index]
    leftovers = leftovers.loc[common_index]
    cost_actual = cost_actual.loc[common_index]

    # Sanity check
    if len(y) < n_future * n_splits:
        raise ValueError("Not enough data to perform the requested number of folds.")

    # Error accumulator
    fold_errors = {model: [] for model in ['sarima', 'prophet', 'bsts']}

    for i in range(1, n_splits + 1):
        start = -n_future * i
        end = -n_future * (i - 1)

        y_train_raw = y.iloc[:start]
        y_test = y.iloc[start:end] if end != 0 else y.iloc[start:]

        offset_i = offset if offset is not None else (abs(y_train_raw.min()) + 1 if use_box_cox else 0)
        y_train = safe_log1p(y_train_raw, offset_i) if use_box_cox else y_train_raw.copy()

        idx = y_train.index.union(y_test.index)
        seas = seasonality.loc[idx]
        dev = deviation_regressor.loc[idx]
        left = leftovers.loc[idx]

        # Scale features (only unified adv and cost)
        train_idx = y.index.get_indexer(y_train.index)
        adv_scaled = scale(adv_actual, train_idx)
        cost_scaled = scale(cost_actual, train_idx)

        # Forecast with all models
        preds_t = {}

        # Pass the same scaled adv to both ad inputs in model functions
        preds_t['sarima'] = sarima_fn(
            adv_scaled,  # unified advertising in place of search/card
            seas, y_train,
            dev, left, cost_scaled,
            p=p, d=d, q=q, P=P, D=D, Q=Q, m=m,
            n_future=n_future, need_summary=False
        ).values

        # Suppress verbose output for prophet
        silent_prophet_fn = suppress_output(prophet_fn)
        preds_t['prophet'] = silent_prophet_fn(
            y_train, adv_scaled,  # unified advertising twice
            seas, dev, left, cost_scaled,
            n_future=n_future
        ).values

        preds_t['bsts'] = bsts_fn(
            y_train, adv_scaled,  # unified advertising twice
            seas, dev, left, cost_scaled,
            forecast_periods=n_future
        )[0].values

        # Inverse transform back to original scale
        preds = {
            model: inverse_transform(vals, offset_i)
            for model, vals in preds_t.items()
        }

        y_true_sum = y_test.sum()
        for model in fold_errors:
            y_pred_sum = preds[model].sum()
            fold_errors[model].append(
                mean_squared_error([y_true_sum], [y_pred_sum])
            )

    # Calculate exponentially weighted average errors
    weights_arr = np.array([alpha * (1 - alpha) ** idx for idx in range(n_splits)])
    weights_arr /= weights_arr.sum()

    exp_err = {model: np.dot(weights_arr, errs) for model, errs in fold_errors.items()}
    inv_err = {model: 1 / err for model, err in exp_err.items()}
    total_inv = sum(inv_err.values())
    model_weights = {model: inv_err[model] / total_inv for model in inv_err}

    return {
        'fold_errors': fold_errors,
        'exp_err': exp_err,
        'model_weights': model_weights
    }


## Расчет ошибки

In [ ]:
import numpy as np
import pandas as pd
import time

def forecast_only(
    y: pd.Series,
    adv: pd.Series,
    seasonality: pd.Series,
    deviation_regressor: pd.Series,
    leftovers: pd.Series,
    cost: pd.Series,
    sarima_fn,
    prophet_fn,
    bsts_fn,
    n_future: int = 31,
    p: int = 1,
    d: int = 0,
    q: int = 1,
    P: int = 0,
    D: int = 0,
    Q: int = 0,
    m: int = 7,
    use_box_cox: bool = True,
    offset: float = None,
    model_weights: dict = None
) -> dict:
    y_train = y.copy()

    offset_f = offset if offset is not None else (abs(y_train.min()) + 1 if use_box_cox else 0)
    y_train_t = np.log1p(y_train + offset_f) if use_box_cox else y_train.copy()

    def scale(series):
        series_std = series.std()
        if series_std == 0:
            series_std = 1
        return (series - series.mean()) / series_std

    adv_scaled = scale(adv)
    cost_scaled = scale(cost)

    future_index = adv.index[-n_future:]  # Предполагаем, что регрессоры уже содержат нужные даты
    seas = seasonality
    dev = deviation_regressor
    left = leftovers

    forecasts_t = {}
    model_times = {}

    # SARIMA
    start_time = time.time()
    forecasts_t['sarima'] = sarima_fn(
        adv_scaled,  # единый регрессор рекламы передаётся дважды
        seas, y_train_t,
        dev, left, cost_scaled,
        p=p, d=d, q=q, P=P, D=D, Q=Q, m=int(m),
        n_future=n_future, need_summary=False
    ).values
    model_times['sarima'] = time.time() - start_time

    # Prophet
    start_time = time.time()
    silent_prophet_fn = suppress_output(prophet_fn)
    forecasts_t['prophet'] = silent_prophet_fn(
        y_train_t, adv_scaled,  # единый регрессор рекламы дважды
        seas, dev, left, cost_scaled,
        n_future=n_future
    ).values
    model_times['prophet'] = time.time() - start_time

    # BSTS
    start_time = time.time()
    forecasts_t['bsts'] = bsts_fn(
        y_train_t, adv_scaled,  # единый регрессор рекламы дважды
        seas, dev, left, cost_scaled,
        forecast_periods=n_future
    )[0].values
    model_times['bsts'] = time.time() - start_time

    # Постобработка прогнозов
    forecasts = {}
    for m_name, vals in forecasts_t.items():
        if use_box_cox:
            pred_vals = np.expm1(vals) - offset_f
        else:
            pred_vals = vals
        pred_vals = np.where(np.isfinite(pred_vals), pred_vals, np.nan)
        pred_vals = np.nan_to_num(pred_vals, nan=0.0, posinf=0.0, neginf=0.0)
        forecasts[m_name] = pred_vals

    if model_weights is None:
        raise ValueError("model_weights must be provided for ensemble forecast")

    ensemble = sum(model_weights[m_name] * forecasts[m_name] for m_name in model_weights)
    ensemble = np.where(np.isfinite(ensemble), ensemble, np.nan)
    ensemble = np.nan_to_num(ensemble, nan=0.0, posinf=0.0, neginf=0.0)

    return {
        'forecast_dates': future_index,
        'model_forecasts': forecasts,
        'ensemble_forecast': ensemble,
        'model_times': model_times
    }
